In [95]:
import pandas as pd
import os

In [96]:
df = pd.read_csv('./raw_data/' + os.listdir('raw_data')[1])

In [97]:
df.columns = [i.lower() for i in df.columns]

In [98]:
df['player'] = [i.split(' ', maxsplit=1)[1] for i in df['player']]

In [99]:
df['date of birth'] = pd.to_datetime(df['date of birth'])

In [100]:
df['weight'] = [int(i.split(' ', maxsplit=1)[0]) for i in df['weight']]

In [101]:
df['height'] = [int(i.split('-')[1].split(' ')[1]) for i in df['height']]

In [102]:
df = df[df['pos'] != 'G']

df['pos'] = ['D' if i.__contains__('D') else 'F' for i in df['pos']]

In [103]:
df['drafted'] = [i.split(' ')[0] for i in df['drafted']]

In [104]:
df['slide cand.'] = [1 if i == '✓' else 0 for i in df['slide cand.']]

In [105]:
df['waivers exempt'] = [1 if i == '✓' else 0 for i in df['waivers exempt']]

In [106]:
df['arb. elig'] = [1 if i == '✓' else 0 for i in df['arb. elig']]

In [107]:
drop_cols = [
    'gp', 'g', 'a', 'p', 'p/gp', '+/-', 'sh', 'sh%', 'toi',
    'w', 'l', 'so', 'gaa', 'sv%', 'arb. elig.1',
    'cap hit', 'salary', 'base salary', 'minors', 's.bonus', 'p.bonus',
    'arb. req', 'exp. year'
]

df = df.drop(drop_cols, axis=1)

In [108]:
type_map = {
    'Standard (1-way)': 'one_way',
    '35+ (1-way)': 'one_way',
    'Standard (2-way)': 'two_way',
    'Entry-Level': 'elc'
}

df['type'] = df['type'].map(type_map)

In [109]:
df['clause'] = df['clause'].fillna('none')

In [110]:
df['cap hit %'] = [float(i.replace('%','')) for i in df['cap hit %']]

In [111]:
df['aav'] = [float(i.replace('$','').replace(',','')) for i in df['aav']]

In [112]:
df.head()

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,arb. elig,type,signing date,signing age,clause,length,signing,expiry,cap hit %,aav
0,Connor McDavid,EDM,22,1997-01-13,Canada,193,185,F,Left,1,...,0,one_way,"Jul 5, 2017",20,none,8,RFA,UFA,16.7,12500000.0
1,Artemi Panarin,NYR,27,1991-10-30,Russia,168,180,F,Right,-,...,0,one_way,"Jul 1, 2019",27,NMC,7,UFA,UFA,14.3,11642857.0
2,Auston Matthews,TOR,21,1997-09-17,United States,223,191,F,Left,1,...,0,one_way,"Feb 5, 2019",21,none,5,RFA,UFA,14.6,11634000.0
3,Erik Karlsson,SJS,29,1990-05-31,Sweden,190,183,D,Right,15,...,0,one_way,"Jun 17, 2019",29,NMC,8,UFA,UFA,14.5,11500000.0
4,John Tavares,TOR,28,1990-09-20,Canada,209,185,F,Left,1,...,0,one_way,"Jul 1, 2018",27,NMC,7,UFA,UFA,13.8,11000000.0


In [113]:
df.loc[0]

player                 Connor McDavid
team                              EDM
age                                22
date of birth     1997-01-13 00:00:00
country                        Canada
weight                            193
height                            185
pos                                 F
handed                           Left
drafted                             1
slide cand.                         0
waivers exempt                      0
arb. elig                           0
type                          one_way
signing date              Jul 5, 2017
signing age                        20
clause                           none
length                              8
signing                           RFA
expiry                            UFA
cap hit %                        16.7
aav                          1.25e+07
Name: 0, dtype: object

# Combine with Career totals prior to signing

In [114]:
players = pd.read_csv('./raw_data/' + os.listdir('raw_data')[0])

In [115]:
import difflib

In [116]:
sample = 'sebastian aho'

difflib.get_close_matches(sample, players['person_fullName'])

['Sebastian Aho', 'Sebastian Aho', 'Sebastien Caron']

In [117]:
player_map = dict(zip(players['person_fullName'], players['person_id']))

In [118]:
df[df['type'] != 'elc'].player.value_counts().head(10)

Matt Roy              2
Nathan Walker         2
Peter Cehlárik        2
Kyle Wood             1
Kevin Shattenkirk     1
William Nylander      1
Jonathan Huberdeau    1
Nic Dowd              1
Laurent Dauphin       1
Cédric Paquette       1
Name: player, dtype: int64

In [119]:
df[df['player']=='Nathan Walker']

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,arb. elig,type,signing date,signing age,clause,length,signing,expiry,cap hit %,aav
1245,Nathan Walker,STL,25,1994-02-07,United Kingdom,186,175,F,Left,89,...,0,two_way,"Jul 1, 2019",25,none,2,UFA,UFA,0.9,700000.0
1250,Nathan Walker,STL,25,1994-02-07,United Kingdom,186,175,F,Left,89,...,0,two_way,"Jul 1, 2019",25,none,2,UFA,UFA,0.9,700000.0


In [120]:
df = df.drop(index=1245, axis=0)

In [121]:
df[df['player']=='Matt Roy']

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,arb. elig,type,signing date,signing age,clause,length,signing,expiry,cap hit %,aav
1246,Matt Roy,LAK,24,1995-03-01,United States,200,185,D,Right,194,...,0,two_way,"Jul 15, 2019",24,none,2,RFA,RFA,0.9,700000.0
1251,Matt Roy,LAK,24,1995-03-01,United States,200,185,D,Right,194,...,0,two_way,"Jul 15, 2019",24,none,2,RFA,RFA,0.9,700000.0


In [122]:
df = df.drop(index=1246, axis=0)

In [123]:
df[df['player']=='Peter Cehlárik']

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,arb. elig,type,signing date,signing age,clause,length,signing,expiry,cap hit %,aav
1299,Peter Cehlárik,BOS,23,1995-08-02,Slovakia,202,188,F,Left,90,...,0,two_way,"Jul 6, 2019",23,none,1,RFA,RFA,0.9,700000.0
1300,Peter Cehlárik,BOS,23,1995-08-02,Slovakia,202,188,F,Left,90,...,0,two_way,"Jul 6, 2019",23,none,1,RFA,RFA,0.9,700000.0


In [124]:
df = df.drop(index=1299, axis=0)

In [125]:
df[df['type'] != 'elc'].player.value_counts().head(10)

Kyle Wood             1
Kevin Shattenkirk     1
Joseph Cramarossa     1
Cal Clutterbuck       1
William Nylander      1
Jonathan Huberdeau    1
Nic Dowd              1
Laurent Dauphin       1
Cédric Paquette       1
André Burakovsky      1
Name: player, dtype: int64

In [126]:
df = df[df['type'] != 'elc']

In [128]:
player_map = dict(zip(players['person_fullName'], players['person_id']))

In [129]:
output = []
for player in df['player']:
    try:
        closest_match = player_map[difflib.get_close_matches(player, players['person_fullName'])[0]]
    except IndexError:
        closest_match = 'not found'
    
    output.append(closest_match)

In [130]:
df['nhl_api_id_number'] = output

In [131]:
df = df[df['nhl_api_id_number'] != 'not found']

In [132]:
df

,player,team,age,date of birth,country,weight,height,pos,handed,drafted,...,type,signing date,signing age,clause,length,signing,expiry,cap hit %,aav,nhl_api_id_number
0,Connor McDavid,EDM,22,1997-01-13,Canada,193,185,F,Left,1,...,one_way,"Jul 5, 2017",20,none,8,RFA,UFA,16.7,12500000.0,8478402
1,Artemi Panarin,NYR,27,1991-10-30,Russia,168,180,F,Right,-,...,one_way,"Jul 1, 2019",27,NMC,7,UFA,UFA,14.3,11642857.0,8478550
2,Auston Matthews,TOR,21,1997-09-17,United States,223,191,F,Left,1,...,one_way,"Feb 5, 2019",21,none,5,RFA,UFA,14.6,11634000.0,8479318
3,Erik Karlsson,SJS,29,1990-05-31,Sweden,190,183,D,Right,15,...,one_way,"Jun 17, 2019",29,NMC,8,UFA,UFA,14.5,11500000.0,8474578
4,John Tavares,TOR,28,1990-09-20,Canada,209,185,F,Left,1,...,one_way,"Jul 1, 2018",27,NMC,7,UFA,UFA,13.8,11000000.0,8475166
5,Drew Doughty,LAK,29,1989-12-08,Canada,202,185,D,Right,2,...,one_way,"Jul 1, 2018",29,NMC,8,UFA,UFA,13.8,11000000.0,8474563
6,Jonathan Toews,CHI,31,1988-04-29,Canada,201,188,F,Left,3,...,one_way,"Jul 9, 2014",26,NMC,8,UFA,UFA,15.2,10500000.0,8473604
8,Patrick Kane,CHI,30,1988-11-19,United States,177,178,F,Left,1,...,one_way,"Jul 9, 2014",26,NMC,8,UFA,UFA,15.2,10500000.0,8474141
10,Anze Kopitar,LAK,31,1987-08-24,Slovenia,225,191,F,Left,11,...,one_way,"Jan 16, 2016",28,NMC,8,UFA,UFA,14.0,10000000.0,8471685
11,Jack Eichel,BUF,22,1996-10-28,United States,200,188,F,Right,2,...,one_way,"Oct 3, 2017",21,none,8,RFA,UFA,13.3,10000000.0,8478403


In [133]:
import requests
import ujson
from pandas.io.json import json_normalize

In [185]:
all_players = pd.DataFrame()
for i in df.index:
    player_id = df.loc[i, 'nhl_api_id_number']
    signing_date = df.loc[i, 'signing date']
    
    
    resp = requests.get(f'https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=yearByYear')
    stats = ujson.loads(resp.text)
    stats = json_normalize(stats['stats'][0]['splits'], sep='_')
    stats = stats[stats['league_id'] == 133]
    stats['player_id'] = player_id
    
    
    start = parse(signing_date).year
    end = start+1
    cutoff = int(str(start)+str(end))

    stats = stats[stats['season'].astype(int) < cutoff]
    
    all_players = pd.concat([all_players, stats], axis=0, ignore_index=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [186]:
all_players

,league_id,league_link,league_name,player_id,season,sequenceNumber,stat_assists,stat_blocked,stat_evenSaves,stat_evenShots,...,stat_shotPct,stat_shots,stat_shotsAgainst,stat_shutouts,stat_ties,stat_timeOnIce,stat_wins,team_id,team_link,team_name
0,133.0,/api/v1/league/133,National Hockey League,8478402,20152016,1,32.0,10.0,NaN,NaN,...,15.2,105.0,NaN,NaN,NaN,849:49,NaN,22.0,/api/v1/teams/22,Edmonton Oilers
1,133.0,/api/v1/league/133,National Hockey League,8478402,20162017,1,70.0,29.0,NaN,NaN,...,12.0,251.0,NaN,NaN,NaN,1732:47,NaN,22.0,/api/v1/teams/22,Edmonton Oilers
2,133.0,/api/v1/league/133,National Hockey League,8478550,20152016,1,47.0,14.0,NaN,NaN,...,16.0,187.0,NaN,NaN,NaN,1481:38,NaN,16.0,/api/v1/teams/16,Chicago Blackhawks
3,133.0,/api/v1/league/133,National Hockey League,8478550,20162017,1,43.0,14.0,NaN,NaN,...,14.7,211.0,NaN,NaN,NaN,1596:38,NaN,16.0,/api/v1/teams/16,Chicago Blackhawks
4,133.0,/api/v1/league/133,National Hockey League,8478550,20172018,1,55.0,19.0,NaN,NaN,...,11.8,228.0,NaN,NaN,NaN,1630:35,NaN,29.0,/api/v1/teams/29,Columbus Blue Jackets
5,133.0,/api/v1/league/133,National Hockey League,8478550,20182019,1,59.0,17.0,NaN,NaN,...,14.7,191.0,NaN,NaN,NaN,1568:40,NaN,29.0,/api/v1/teams/29,Columbus Blue Jackets
6,133.0,/api/v1/league/133,National Hockey League,8479318,20162017,1,29.0,61.0,NaN,NaN,...,14.3,279.0,NaN,NaN,NaN,1445:45,NaN,10.0,/api/v1/teams/10,Toronto Maple Leafs
7,133.0,/api/v1/league/133,National Hockey League,8479318,20172018,1,29.0,61.0,NaN,NaN,...,18.2,187.0,NaN,NaN,NaN,1123:57,NaN,10.0,/api/v1/teams/10,Toronto Maple Leafs
8,133.0,/api/v1/league/133,National Hockey League,8479318,20182019,1,36.0,60.0,NaN,NaN,...,14.7,251.0,NaN,NaN,NaN,1261:44,NaN,10.0,/api/v1/teams/10,Toronto Maple Leafs
9,133.0,/api/v1/league/133,National Hockey League,8474578,20092010,1,21.0,48.0,NaN,NaN,...,4.5,112.0,NaN,NaN,NaN,1206:48,NaN,9.0,/api/v1/teams/9,Ottawa Senators


In [215]:
stats = all_players

In [216]:
keep_cols = [
    'player_id', 'season', 'stat_assists', 'stat_blocked', 'stat_evenTimeOnIce', 'stat_faceOffPct',
    'stat_gameWinningGoals', 'stat_games', 'stat_goals', 'stat_hits',
    'stat_overTimeGoals', 'stat_penaltyMinutes', 'stat_pim',
    'stat_plusMinus', 'stat_points', 'stat_powerPlayGoals',
    'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_shifts',
    'stat_shortHandedGoals', 'stat_shortHandedPoints',
    'stat_shortHandedTimeOnIce', 'stat_shotPct', 'stat_shots',
    'stat_timeOnIce',
]

stats = stats.filter(items=keep_cols)

In [217]:
stats.columns = [i.replace('stat_', '') for i in stats.columns]

In [218]:
stats.columns

Index(['player_id', 'season', 'assists', 'blocked', 'evenTimeOnIce',
       'faceOffPct', 'gameWinningGoals', 'games', 'goals', 'hits',
       'overTimeGoals', 'penaltyMinutes', 'pim', 'plusMinus', 'points',
       'powerPlayGoals', 'powerPlayPoints', 'powerPlayTimeOnIce', 'shifts',
       'shortHandedGoals', 'shortHandedPoints', 'shortHandedTimeOnIce',
       'shotPct', 'shots', 'timeOnIce'],
      dtype='object')

In [219]:
stats.isnull().sum()

player_id                0
season                   0
assists                 17
blocked                 79
evenTimeOnIce           79
faceOffPct              79
gameWinningGoals        17
games                    0
goals                   17
hits                    79
overTimeGoals           17
penaltyMinutes          17
pim                     17
plusMinus               17
points                  17
powerPlayGoals          17
powerPlayPoints         17
powerPlayTimeOnIce      79
shifts                  79
shortHandedGoals        17
shortHandedPoints       17
shortHandedTimeOnIce    79
shotPct                 18
shots                   17
timeOnIce               62
dtype: int64

In [220]:
def toi_to_float(x):
    if x is not np.nan:
        split = x.split(':')
        split[1] = str(int(split[1])/60)
        return float(split[0]) + float(split[1])
    else:
        return 0

toi_cols = [
    'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'timeOnIce'
]

for col in toi_cols:
    stats[col] = stats[col].apply(toi_to_float)

In [221]:
import numpy as np

In [236]:
player_stats = stats.groupby('player_id').sum().reset_index()
player_stats = player_stats.rename(columns={'player_id': 'nhl_api_id_number'})

player_stats['nhl_api_id_number'] = player_stats['nhl_api_id_number'].astype(int)
df['nhl_api_id_number'] = df['nhl_api_id_number'].astype(int)

df = df.merge(player_stats, how='left', on='nhl_api_id_number')

In [238]:
df.loc[0]

player                       Connor McDavid
team                                    EDM
age                                      22
date of birth           1997-01-13 00:00:00
country                              Canada
weight                                  193
height                                  185
pos                                       F
handed                                 Left
drafted                                   1
slide cand.                               0
waivers exempt                            0
arb. elig                                 0
type                                one_way
signing date                    Jul 5, 2017
signing age                              20
clause                                 none
length                                    8
signing                                 RFA
expiry                                  UFA
cap hit %                              16.7
aav                                1.25e+07
nhl_api_id_number               

In [239]:
df.to_csv('./model_data/salaries_with_stats.csv', index=False)

In [240]:
df.describe()

,age,weight,height,slide cand.,waivers exempt,arb. elig,signing age,length,cap hit %,aav,...,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,shifts,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,shotPct,shots,timeOnIce
count,734.000000,734.000000,734.00000,734.0,734.000000,734.0,734.000000,734.000000,734.000000,7.340000e+02,...,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,27.268392,199.171662,185.23842,0.0,0.047684,0.0,26.036785,3.384196,3.906267,2.917735e+06,...,10.676309,30.526171,445.610101,5662.933884,1.404959,2.713499,300.615358,42.021309,482.061983,4403.828627
std,3.428897,15.383163,5.61173,0.0,0.213242,0.0,3.078333,2.458290,3.491113,2.495357e+06,...,20.372833,58.127197,687.442507,5905.469800,2.909424,5.068144,489.682942,40.694945,593.894704,4761.126913
min,21.000000,150.000000,165.00000,0.0,0.000000,0.0,20.000000,1.000000,0.800000,6.666660e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,189.000000,183.00000,0.0,0.000000,0.0,24.000000,2.000000,0.900000,7.037500e+05,...,0.000000,0.000000,11.904167,1233.500000,0.000000,0.000000,7.083333,12.225000,89.250000,935.054167
50%,27.000000,199.000000,185.00000,0.0,0.000000,0.0,25.000000,2.000000,2.500000,1.987500e+06,...,2.000000,7.000000,140.800000,3831.500000,0.000000,1.000000,100.425000,31.050000,264.000000,2902.316667
75%,29.000000,209.750000,188.00000,0.0,0.000000,0.0,28.000000,5.000000,6.475000,4.750000e+06,...,12.000000,34.000000,535.191667,7853.250000,1.000000,3.000000,387.020833,58.700000,635.750000,5888.675000
max,42.000000,260.000000,206.00000,0.0,1.000000,0.0,42.000000,14.000000,16.800000,1.250000e+07,...,166.000000,722.000000,4655.866667,41148.000000,22.000000,50.000000,5296.550000,225.000000,5157.000000,35782.116667


In [241]:
df.columns

Index(['player', 'team', 'age', 'date of birth', 'country', 'weight', 'height',
       'pos', 'handed', 'drafted', 'slide cand.', 'waivers exempt',
       'arb. elig', 'type', 'signing date', 'signing age', 'clause', 'length',
       'signing', 'expiry', 'cap hit %', 'aav', 'nhl_api_id_number', 'assists',
       'blocked', 'evenTimeOnIce', 'faceOffPct', 'gameWinningGoals', 'games',
       'goals', 'hits', 'overTimeGoals', 'pim', 'plusMinus', 'points',
       'powerPlayGoals', 'powerPlayPoints', 'powerPlayTimeOnIce', 'shifts',
       'shortHandedGoals', 'shortHandedPoints', 'shortHandedTimeOnIce',
       'shotPct', 'shots', 'timeOnIce'],
      dtype='object')

In [254]:
keep_cols = [
    'player' ,'nhl_api_id_number', 'team', 'cap hit %', 'aav', 'length',
    'weight', 'height','pos', 'handed', 'drafted', 'slide cand.', 'waivers exempt',
    'arb. elig', 'type', 'signing age', 'signing', 'expiry', 'goals', 'assists',
    'points', 'blocked', 'evenTimeOnIce', 'faceOffPct', 'gameWinningGoals', 'games',
    'hits', 'overTimeGoals', 'pim', 'plusMinus', 
    'powerPlayGoals', 'powerPlayPoints', 'powerPlayTimeOnIce', 'shifts',
    'shortHandedGoals', 'shortHandedPoints', 'shortHandedTimeOnIce',
    'shotPct', 'shots', 'timeOnIce'
]

data = df[keep_cols]

In [255]:
data.head(2)

,player,nhl_api_id_number,team,cap hit %,aav,length,weight,height,pos,handed,...,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,shifts,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,shotPct,shots,timeOnIce
0,Connor McDavid,8478402,EDM,16.7,12500000.0,8,193,185,F,Left,...,6.0,41.0,382.250000,3028.0,1.0,2.0,100.966667,27.2,356.0,2582.600000
1,Artemi Panarin,8478550,NYR,14.3,11642857.0,7,168,180,F,Right,...,30.0,80.0,944.883333,7328.0,0.0,0.0,8.766667,57.2,817.0,6277.516667


In [262]:
data.loc[0]

player                  Connor McDavid
nhl_api_id_number              8478402
team                               EDM
cap hit %                         16.7
aav                           1.25e+07
length                               8
weight                             193
height                             185
pos                                  F
handed                            Left
drafted                              1
slide cand.                          0
waivers exempt                       0
arb. elig                            0
type                           one_way
signing age                         20
was_RFA_at_signing                   1
is_RFA_at_expiry                     0
goals                               46
assists                            102
points                             148
blocked                             39
evenTimeOnIce                  2099.38
faceOffPct                       84.41
gameWinningGoals                    11
games                    

In [256]:
data['signing'] = data['signing'].map({'RFA':1,'UFA':0})
data['expiry'] = data['expiry'].map({'RFA':1,'UFA':0})

cols = {
    'signing':'was_RFA_at_signing',
    'expiry': 'is_RFA_at_expiry'
}

data = data.rename(columns=cols)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [263]:

data = pd.concat([data.drop('pos', axis=1), 
                  pd.get_dummies(data['pos'], 
                                 drop_first=True, 
                                 prefix='position'
                                )
                 ],
                 axis=1,
                 sort=False
                )

data = pd.concat([data.drop('handed', axis=1), 
                  pd.get_dummies(data['handed'], 
                                 drop_first=True, 
                                 prefix='shoots'
                                )
                 ],
                 axis=1,
                 sort=False
                )

data = pd.concat([data.drop('type', axis=1), 
                  pd.get_dummies(data['type'], 
                                 drop_first=True, 
                                 prefix='contract_is'
                                )
                 ],
                 axis=1,
                 sort=False
                )

data

,player,nhl_api_id_number,team,cap hit %,aav,length,weight,height,drafted,slide cand.,...,shifts,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,shotPct,shots,timeOnIce,position_F,shoots_Right,contract_is_two_way
0,Connor McDavid,8478402,EDM,16.7,12500000.0,8,193,185,1,0,...,3028.0,1.0,2.0,100.966667,27.2,356.0,2582.600000,1,0,0
1,Artemi Panarin,8478550,NYR,14.3,11642857.0,7,168,180,-,0,...,7328.0,0.0,0.0,8.766667,57.2,817.0,6277.516667,1,1,0
2,Auston Matthews,8479318,TOR,14.6,11634000.0,5,223,191,1,0,...,4982.0,0.0,0.0,5.083333,47.2,717.0,3831.433333,1,0,0
3,Erik Karlsson,8474578,SJS,14.5,11500000.0,8,190,183,15,0,...,18607.0,1.0,5.0,836.666667,62.2,2014.0,17576.650000,0,1,0
4,John Tavares,8475166,TOR,13.8,11000000.0,7,209,185,1,0,...,14984.0,3.0,6.0,249.716667,117.8,2110.0,13395.250000,1,0,0
5,Drew Doughty,8474563,LAK,13.8,11000000.0,8,202,185,2,0,...,23289.0,2.0,6.0,1938.366667,62.7,1668.0,20218.183333,0,1,0
6,Jonathan Toews,8473604,CHI,15.2,10500000.0,8,201,188,3,0,...,12359.0,8.0,19.0,719.283333,106.5,1295.0,9597.850000,1,0,0
7,Patrick Kane,8474141,CHI,15.2,10500000.0,8,177,178,1,0,...,12029.0,0.0,1.0,34.383333,83.4,1540.0,9932.466667,1,0,0
8,Anze Kopitar,8471685,LAK,14.0,10000000.0,8,225,191,11,0,...,19204.0,10.0,16.0,1378.666667,123.2,1959.0,15904.583333,1,0,0
9,Jack Eichel,8478403,BUF,13.3,10000000.0,8,200,188,2,0,...,3139.0,0.0,0.0,45.133333,19.7,487.0,2763.983333,1,1,0


In [269]:
def drafted(x):
    x = str(x)
    return x.replace('-', '218')

data['drafted']=data['drafted'].apply(drafted)

217

In [270]:
data.to_csv('./model_data/salaries_with_stats.csv', index=False)